In [42]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
import heapq
from keybert import KeyBERT
import nltk
import math
from pythainlp.tag import tag_provinces

In [14]:
class spyder:
    def __init__( self ,target_links,base_url ):
        self.base_url = base_url
        self.target_links = target_links
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,0,set())
        check_domain =  self.check_domain(self.base_url,crawl) 
        check_not_domain = self.check_not_domain(self.base_url,crawl)
        check_ref = self.check_ref(check_not_domain,self.target_links)
        return check_domain,check_ref
    
    def crawl(self,url, depth,visited):
        if depth < 2 :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    #if link.startswith(url):
                    self.crawl(link,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [60]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        if self.keyword_result == {}:
            return {'NONE':0}
        else:
            return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=10)
        if self.summarize_result == []:
            return 'NONE'
        else:
            return self.summarize_result
    def get_tokenize(self):
        self.word_token = word_tokenize(self.sentence, engine="newmm")
        return self.word_token
    def get_location(self):
        self.location_Result = self.location()
        self.location_counts = {}
        for location, _ in self.location_Result:
            if location in self.location_counts:
                self.location_counts[location] += 1
            else:
                self.location_counts[location] = 1
        return self.location_counts
    def location(self):
        self.data = self.get_tokenize()
        self.location_value = tag_provinces(self.data)
        self.Result_location = filtered_input = [entry for entry in self.location_value if entry[1] == 'B-LOCATION']
        return self.Result_location


In [15]:
class Eng:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def make_sentence(self,list_word):
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def tokenize(self):
        self.sentence_list = nltk.sent_tokenize(self.sentence)
        return self.sentence_list
    def create_formatted_article(self):
        # Removing special characters and digits
        self.get_sentence()
        self.formatted_article_text = re.sub('[^a-zA-Z]', ' ', self.sentence_result )
        self.formatted_article_text = re.sub(r'\s+', ' ', self.formatted_article_text)
        return self.formatted_article_text
    def find_word_frequencies(self):
        self.tokenize()
        self.create_formatted_article()
        stopwords = nltk.corpus.stopwords.words('english')
        self.word_frequencies = {}
        for word in nltk.word_tokenize(self.formatted_article_text):
            if word not in stopwords:
                if word not in self.word_frequencies.keys():
                    self.word_frequencies[word] = 1
                else:
                    self.word_frequencies[word] += 1
        return self.word_frequencies
    def find_maximum_frequncy(self):
        self.find_word_frequencies()
        maximum_frequncy = max(self.word_frequencies.values())
        for word in self.word_frequencies.keys():
            self.word_frequencies[word] = (self.word_frequencies[word]/maximum_frequncy)
    def find_sentence_scores(self):
        self.sentence_scores = {}
        self.tokenize()
        self.find_word_frequencies()
        for sent in self.sentence_list:
            for word in nltk.word_tokenize(sent.lower()):
                if word in self.word_frequencies.keys():
                    if len(sent.split(' ')) < 30:
                        if sent not in self.sentence_scores.keys():
                            self.sentence_scores[sent] = self.word_frequencies[word]
                        else:
                            self.sentence_scores[sent] += self.word_frequencies[word]
        return self.sentence_scores
    def summary_sentences(self):
        self.find_sentence_scores()
        self.summary_sentences = heapq.nlargest(5, self.sentence_scores, key=self.sentence_scores.get)
        summary = ' '.join(self.summary_sentences)
        return self.summary_sentences
    def find_keyword(self):
        self.get_sentence()
        kw_model = KeyBERT()
        keywords = kw_model.extract_keywords(self.sentence_result)
        return kw_model.extract_keywords(self.sentence_result, keyphrase_ngram_range=(1, 1), stop_words=None)

In [59]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = 'จ.'+max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    try:
      title_tag = summarize_article[0]
    except:
      title_tag = 'NONE'
    
  return p_tag, title_tag, keyword,location

In [17]:
target_links = {'https://www.thairath.co.th/news':0,'https://news.kapook.com':0,'www.bbc.com/news':0}

for i in target_links:
    get_link = spyder(target_links,i)
    ref = get_link.get_check_ref()
print(ref)

{'https://www.thairath.co.th/news': 0, 'https://news.kapook.com': 0}


In [61]:
a='https://www.thairath.co.th/news/royal/2617245'
response = requests.get(a)
soup = BeautifulSoup(response.text, 'html.parser')
try:
    title_tag = soup.find('title').text
except:
    title_tag = soup.find('title')
p_tags = soup.find_all('p')
p_list =[]
for p in p_tags:
    if p.string != None:
        p_list.append(unicodedata.normalize("NFKD", p.string))
    if len(p_list) == 0:
        p_list.append('ไม่พบข้อความในเว็บนี้')
  
p_tag = "".join(p_list)
thai_nlp = Thai(p_list)
keyword=thai_nlp.keyword
print(keyword)
keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
print(keyword)

{'NONE': 0}
{'NONE': 0}


In [62]:
doc=[]
for i in target_links:
    web_spyder=spyder(target_links,i)
    domain_links =web_spyder.get_check_domain()
    for j in domain_links:
        print(j)
        j.replace(" ", "")
        d=dict()    
        body, title, keyword,location = scrape_tags(j)
        d['link']= j
        d['title'] = title
        d['body']=body
        d['location']=location
        d['keyword'] = keyword
        for k in target_links:
            if j.startswith(k):
                d['ref'] = target_links[k]
        doc.append(d)
        print(d)

https://www.thairath.co.th/news/tech/2625120
{'link': 'https://www.thairath.co.th/news/tech/2625120', 'title': 'Bard แชตบอตของกูเกิลให้ข้อมูลที่ผิดพลาดในช่วงเดโม', 'body': 'อย่างไรก็ดี ในช่วงของรอบสาธิตการใช้งาน ผลปรากฏว่า Bard กลับให้ข้อมูลที่ผิดพลาดในประเด็นเรื่องของกล้องโทรทรรศน์ของเจมส์ เวบบ์ ซึ่ง Bard บอกว่า กล้องของเจมส์ เวบบ์ เพิ่งถ่ายภาพนอกระบบสุริยะได้เป็นครั้งแรก ทั้งที่ในความเป็นจริงแล้วสามารถถ่ายภาพนอกระบบสุริยะทําได้สําเร็จตั้งแต่ปี 2004จากความผิดพลาดในครั้งนี้ของ Bard สะท้อนให้เห็นปัญหาอันใหญ่หลวงของบริการแชตบอต และเป็นการยืนยันอีกครั้งหนึ่งว่า ข้อมูลที่ปรากฏบนแชตบอตไม่ได้เป็นข้อมูลที่ถูกต้อง 100 เปอร์เซ็นต์เสมอไป และทําให้ผู้ใช้งานทุกคนอาจต้องมีการตรวจสอบทุกครั้งที่มีการใช้งานแชตบอตทั้งนี้ แชตบอตไม่ว่าจะเป็น ChatGPT หรือ Bard ได้มีการบ่งบอกคําเตือนอย่างชัดเจนว่า การให้ข้อมูลอาจมีความผิดพลาดได้ ผู้ใช้งานจึงควรใช้งานแชตบอตนี้ด้วยความระมัดระวังหลังจากเกิดประเด็นนี้ กูเกิล ยืนยันว่าจะเน้นให้เห็นถึงความสําคัญของกระบวนการตรวจสอบอันเข้มงวด โดยในสัปดาห์นี้จะมีโปรแกรมที่มีชื่อว่า

In [122]:
import sqlite3

# Connect to SQLite3 database
conn = sqlite3.connect('inverted_index.db')
cursor = conn.cursor()

# Create tables for words, documents, and word frequencies
conn.execute('''
CREATE TABLE words (
    ID INTEGER PRIMARY KEY,
    Word TEXT NOT NULL UNIQUE
);
''')

conn.execute('''
CREATE TABLE documents (
    ID INTEGER PRIMARY KEY,
    Link TEXT NOT NULL UNIQUE ,
    Title TEXT,
    Body TEXT,
    Location TEXT,
    Ref INTEGER
);
''')

conn.execute('''
CREATE TABLE word_frequencies (
    Word_ID INTEGER ,
    Doc_ID INTEGER ,
    Frequency INTEGER NOT NULL,
    TF_IDF REAL NOT NULL,
    PRIMARY KEY (word_id, doc_id),
    FOREIGN KEY (word_id) REFERENCES words(id),
    FOREIGN KEY (doc_id) REFERENCES documents(id)
);
''')

In [123]:
for i in doc:
  conn.execute('''INSERT INTO documents (Link, Title, Body, Location, Ref) VALUES (?, ?, ?, ?, ?);''', (i['link'], i['title'], i['body'],i['location'],i['ref']))
  doc_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
  doc_len = sum(i['keyword'].values())
  for j in i['keyword'].keys():
    word_id = conn.execute("SELECT id FROM words WHERE word = ?", (j,)).fetchone()
    if not word_id:
      conn.execute("INSERT INTO words (word) VALUES (?)", (j,))
      word_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
    else:
      word_id = word_id[0]

    try:
      tf = i['keyword'][j] / doc_len
    except:
      tf = 0
    try:
      idf = math.log(len(doc) / len(conn.execute('''SELECT DISTINCT doc_id FROM word_frequencies WHERE word_id = ?;''', (word_id,)).fetchall()))
    except:
      idf = 0
    tfidf = tf * idf
    conn.execute('''INSERT INTO word_frequencies (word_id, doc_id, Frequency, TF_IDF) VALUES (?, ?, ?, ?);''', (word_id, doc_id, i['keyword'][j], tfidf))
    
conn.commit()
#conn.close()